In [1]:
%matplotlib inline

import numpy as np
import random
import matplotlib.pyplot as plt

from sklearn.datasets import make_gaussian_quantiles
from datetime import datetime
from scipy.sparse import csr_matrix
from sklearn import datasets, svm, metrics

In [99]:
n_events = 300   # Each event actually produces one gamma ray shower and one cosmic ray shower as per the for loop below,
n_pmt = 25     # meaning the total number of events will be twice the input value n_events

targets = np.arange(0, 2, 1)
increment_targets = np.arange(0, 2, 1)
all_images = np.zeros((n_events * 2, 180, 180))
print(all_images.shape)

for i in range(n_events):
    gamma, ray_1 = make_gaussian_quantiles(mean = (8,8), cov = 1,
                                             n_samples = n_pmt, n_features = 2,
                                             n_classes = 1, random_state = i) 

    cosmic, ray_2 = make_gaussian_quantiles(mean = (7, 7), cov = 1,
                                             n_samples = n_pmt, n_features = 2,
                                             n_classes = 1, random_state = i + 2) 

    # This is slightly confusing, make_gaussian_quantiles defines the variable x as an n x 2 matrix of x, y coordinates
    # Now, I'm simply separating that matrix into two arrays: one array of x coordinates, one array of y coordinates

    gammax_array = gamma[:, 0]
    gammay_array = gamma[:, 1]

    cosmicx_array = cosmic[:, 0]
    cosmicy_array = cosmic[:, 1]

    detection = np.arange(0, n_pmt, 1)

    # At some point, ask Udara on what grid system HAWC is built.  Hopefully, the origin is not in the center. 
    # I think I can define it myself... 

    for j in range(n_pmt):
        detection[j] = 1
        gammax_array[j] = "{0:.1f}".format(gammax_array[j])
        gammay_array[j] = "{0:.1f}".format(gammay_array[j])
        cosmicx_array[j] = "{0:.1f}".format(cosmicx_array[j])
        cosmicy_array[j] = "{0:.1f}".format(cosmicy_array[j])

    pixels = np.arange(0, 18, 0.1)
    gammax_indeces = gammax_array*10
    gammay_indeces = gammay_array*10
    cosmicx_indeces = cosmicx_array*10
    cosmicy_indeces = cosmicy_array*10
    
    all_indeces = [gammax_indeces, gammay_indeces, cosmicx_indeces, cosmicy_indeces]
    
    for k in range(n_pmt):
        for l, idx in zip(range(4), all_indeces):
            if idx[k] > 179:
                idx[k] = 179.
            elif idx[k] < 0.:
                idx[k] = 0.
        for m, idx2 in zip(range(n_pmt), all_indeces):
            if idx2[k] == idx2[m]:
                idx2[k] += 1

    gammax_min, gammax_max = gammax_indeces.min() - 10, gammax_indeces.max() + 10
    gammay_min, gammay_max = gammay_indeces.min() - 10, gammay_indeces.max() + 10
    cosmicx_min, cosmicx_max = cosmicx_indeces.min() - 10, cosmicx_indeces.max() + 10
    cosmicy_min, cosmicy_max = cosmicy_indeces.min() - 10, cosmicy_indeces.max() + 10
    
    # Come back to this, these arrays should probably be one and the same for the purpose of training the algorithm
    gamma_image = csr_matrix((detection, (gammax_indeces, gammay_indeces)), shape = (len(pixels), len(pixels))).toarray() #These parentheses are important
    cosmic_image = csr_matrix((detection, (cosmicx_indeces, cosmicy_indeces)), shape = (len(pixels), len(pixels))).toarray()
    
    all_images[2 * i] = gamma_image
    all_images[(2 * i) + 1] = cosmic_image
    
    if i == 0: 
        i = 0
    else:
        targets = np.concatenate((targets, increment_targets))
        
images_targets = list(zip(all_images, targets))

(600, 180, 180)


In [100]:
Just to make sure everything seems to be working correctly
for n, (image, target) in enumerate(images_targets):
    plt.figure(figsize = (8, 8))
    plt.imshow(image, cmap = plt.cm.gray, interpolation = 'nearest')
    plt.autoscale(enable = True, axis = 'both')
    plt.title('Target: %x' %target)
    plt.show()

In [101]:
# Reshape the image to go through classifier
data = all_images.reshape((n_events * 2, -1))

print(data.shape)

# Classifier
clf = svm.SVC(gamma = 0.001)

# Train the algorithm
clf.fit(data[:n_events], targets[:n_events])

(600, 32400)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [102]:
expected = targets[:n_events]
predicted = clf.predict(data[n_events:])

# expected = digits.target[n_samples // 2:]
# predicted = classifier.predict(data[n_samples // 2:])

print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
             precision    recall  f1-score   support

          0       0.95      0.99      0.97       150
          1       0.99      0.95      0.97       150

avg / total       0.97      0.97      0.97       300


Confusion matrix:
[[149   1]
 [  8 142]]
